# 특징추출

In [1]:
import cv2

In [ ]:
def img_pro(func, img, *args, output=True, **kwargs):
    img_ = func(img, *args, **kwargs)
    if output:
        cv2.imshow("Image",img_)
        cv2.waitKey()
        cv2.destroyAllWindows()
    else:
        return img_

In [ ]:
def img_pro2(func, img, *args, show=True, win=False, **kwargs):
    img_=func(img, *args, **kwargs)
    if show:
        if win:
            cv2.imshow("Image",img_)
            cv2.waitKey()
            cv2.destroyAllWindows()
        else:
            fig, axes = plt.subplots(1,2)
            axes[0].imshow(img, cmap="gray")
            axes[0].axis("off"); axes[0].set_title("origin")
            axes[1].imshow(img_, cmap="gray", interpolation=None)
            axes[1].axis("off"); axes[1].set_title("target")
    else:
           return img_

## Contour

### labeling

In [2]:
def labelset(img, xs, ys, label):
    height, width = img.shape
    img[ys, xs] = label
    while True:
        cnt = 0
        
        for y in range(1, height-1):
            for x in range(1, width-1):
                if img[y,x] == label:
                    if img[y, x+1]  == 255:
                        img[y, x+1]  = label
                        cnt = cnt+1
                    if img[y-1, x+1]  == 255:
                        img[y-1, x+1]  = label
                        cnt = cnt+1
                    if img[y-1, x]  == 255:
                        img[y-1, x]  = label
                        cnt = cnt+1
                    if img[y-1, x-1]  == 255:
                        img[y-1, x-1]  = label
                        cnt = cnt+1
                    if img[y, x-1]  == 255:
                        img[y, x-1]  = label
                        cnt = cnt+1
                    if img[y+1, x-1]  == 255:
                        img[y+1, x-1]  = label
                        cnt = cnt+1
                    if img[y+1, x]  == 255:
                        img[y+1, x]  = label
                        cnt = cnt+1
                    if img[y+1, x+1]  == 255:
                        img[y+1, x+1]  = label
                        cnt = cnt+1
                        
        if cnt==0:
            return (0,img)
    return (1,img)

In [12]:
def labeling(img_in, cnt):
    height, width = img_in.shape
    img_label = img_in.copy()
    
    label = 100
    
    for y in range(1, height-1):
            for x in range(1, width-1):
                if img_label[y,x] == 255:
                    if label >= 255:
                        print("Error! too many labels")
                        return -1
                    
                    _, img_label = labelset(img_label, x, y , label)
                    label = label +1
                
    cnt = label -100
    
    return img_label,cnt

In [13]:
img = cv2.imread('shape.png',0)
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
img = cv2.imread('shape.png',0)
ret, bin_img = cv2.threshold(img,128,255,cv2.THRESH_BINARY)
labeled_img, cnt = labeling(bin_img, 100)
cnt

3

In [16]:
labeled_img[80,100] , labeled_img[160,190], labeled_img[160,60]

(100, 101, 102)

In [18]:
cv2.imshow('image',labeled_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [19]:
cv2.imwrite('shape_labeled.png', labeled_img)

True

In [20]:
img = cv2.imread('sample.png',0)
ret, bin_img = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY)
cnt, labled_img = cv2.connectedComponents(bin_img)

In [21]:
labeled_img[80,100] , labeled_img[160,190], labeled_img[160,60]

(100, 101, 102)

In [22]:
labeled_img.dtype

dtype('uint8')

In [23]:
labeled_img = labeled_img * 50

In [25]:
cv2.imshow('image',labeled_img.astype(np.uint8))
cv2.waitKey(0)
cv2.destroyAllWindows()

### 면적과 중심 위치

In [26]:
def calc_size(image_label, label):
    tx, ty = 0, 0
    cv, cy = 0, 0
    total = 0
    height, width = image_label.shape
    for y in range(height):
        for x in range(width):
            if image_label[y,x] == label:
                tx = tx + x
                ty = ty + y
                total = total +1
    if total == 0:
        return 0
    cx = int(tx/total)
    cy = int(ty/total)
    
    return (cx, cy, total)

In [27]:
img = cv2.imread('shape.png', 0)
ret, bin_img = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY)
labeled_img, cnt = labeling(bin_img, 100)
for i in range(cnt):
    cx, cy, area = calc_size(labeled_img, i+100)
    print(cx, cy, area)

114 79 4880
192 168 1108
81 163 2759


### 주위 길이

In [37]:
def trace(image_label, xs, ys):
    length = 0
    x, y =xs, ys
    no = image_label[y,x+1]
    vec = 5
    
    while True:
        if (x==xs) & (y==ys) & (length !=0):
            return length
        image_label[y,x] = 255
        
        if vec ==3:
            if (image_label[y, x+1] != no) & (image_label[y-1, x+1] == no):
                x=x+1
                y= y
                length = length+1
                vec = 0
                continue
            else:
                vec = 4
        if vec ==4:
            if (image_label[y-1, x+1] != no) & (image_label[y-1, x] == no):
                x=x+1
                y= y-1
                length = length+np.sqrt(2)
                vec = 1
                continue
            else:
                vec = 5
        if vec ==5:
            if (image_label[y-1, x] != no) & (image_label[y-1, x-1] == no):
                x=x
                y= y-1
                length = length+1
                vec = 2
                continue
            else:
                vec = 6
        if vec ==6:
            if (image_label[y-1, x-1] != no) & (image_label[y, x-1] == no):
                x=x-1
                y= y-1
                length = length+np.sqrt(2)
                vec = 3
                continue
            else:
                vec = 7
        if vec ==7:
            if (image_label[y, x-1] != no) & (image_label[y+1, x-1] == no):
                x=x-1
                y= y
                length = length+1
                vec = 4
                continue
            else:
                vec = 0
        if vec ==0:
            if (image_label[y+1, x-1] != no) & (image_label[y+1, x] == no):
                x=x-1
                y= y+1
                length = length+np.sqrt(2)
                vec = 5
                continue
            else:
                vec = 1
        if vec ==1:
            if (image_label[y+1, x] != no) & (image_label[y+1, x+1] == no):
                x=x
                y= y+1
                length = length+1
                vec = 6
                continue
            else:
                vec = 2
        if vec ==2:
            if (image_label[y+1, x+1] != no) & (image_label[y, x+1] == no):
                x=x+1
                y= y+1
                length = length+np.sqrt(2)
                vec = 7
                continue
            else:
                vec = 3

In [38]:
def calc_length(image_label, label):
    height, width = image_label.shape
    for y in range(height):
        for x in range(width):
            if image_label[y,x] == label:
                length = trace(image_label, x-1, y)
                return length

In [39]:
def features(image_label, cnt):
    areas = []
    centers = []
    lengths = []
    ratios = []
    
    for i in range(cnt):
        cx, cy, area = calc_size(image_label, i+100)
        centers.append((cx,cy))
        areas.append(area)
        print(i+100, cx, cy, area, end=" ")
        length = calc_length(image_label, i+100)
        lengths.append(length)
        ratio = 4*np.pi*area / (length*length)
        ratios.append(ratio)
        print(length, ratio)
        
    return (areas, centers, lengths, ratios)

In [40]:
img = cv2.imread('shape.png', 0)
ret, bin_img = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY)
labeled_img, cnt = labeling(bin_img, 100)
feat = features(labeled_img, cnt)

100 114 79 4880 263.7645019878176 0.8814486001080429
101 192 168 1108 168.2253967444163 0.49200158512447256
102 81 163 2759 241.6568542494924 0.5936949146124189
